# Static yaw stability derivative

This section computes yaw static stability derivative $C_{n_{\beta}}$ for the example airplane. The thrust effects are neglected for this analysis. Using equation 16.42 in Raymer, yaw static stability derivative can be written as:

$$
    C_{n_{\beta}} = C_{n_{\beta_w}} + C_{n_{\beta_{fus}}} + C_{n_{\beta_v}}.
$$

The first, second, and third term represent yawing moment contribution from wing, fuselage, and vertical tail, respectively. Following sub-sections compute these component-level derivatives and estimates $C_{n_{\beta}}$ for the airplane.

## Wing

The wing contribution towards yawing moment can be computed as (equation 16.44, Raymer):

$$
    C_{n_{\beta_w}} = \frac{C_L^2}{4\pi A},
$$

where $A$ is the wing aspect ratio and $C_L$ is the lift coefficient. Note that sweep effects are not included in the above equation since example airplane has zero quarter-chord sweep. Below code block computes $C_{n_{\beta_w}}$ at three different flight conditions:

In [14]:
import numpy as np

# Parameters
A = 8
mac = 4.3 # ft
CL_landing = 2.2
CL_takeoff = 1.8
CL_cruise = 0.38

Cn_beta_wing_landing = CL_landing**2/np.pi/A/4
Cn_beta_wing_takeoff = CL_takeoff**2/np.pi/A/4
Cn_beta_wing_cruise = CL_cruise**2/np.pi/A/4

print(f"Yaw static stability derivative for wing (1/rad):")
print(f"landing: {Cn_beta_wing_landing:.4f}")
print(f"takeoff: {Cn_beta_wing_takeoff:.4f}")
print(f"cruise: {Cn_beta_wing_cruise:.4f}")

Yaw static stability derivative for wing (1/rad):
landing: 0.0481
takeoff: 0.0322
cruise: 0.0014


## Fuselage

The fuselage contribution towards yawing moment can be computed as (equation 16.50, Raymer):

$$
    C_{n_{\beta_{fus}}} = -1.3\frac{V}{bS_w} \bigg( \frac{D_f}{W_f} \bigg),
$$

where $V$ is the volume of the fuselage, $b$ is the wing span, $S_w$ is the wing reference area, $D_f$ is the height of the fuselage, and $W_f$ is the width of the fuselage. The $V$ for the example airplane is computed using OpenVSP model. Below block computes $C_{n_{\beta_{fus}}}$:

In [16]:
# Parameters
b = 33 # ft
Sw = 134 # sq ft
V = 417 # cu ft, vsp model
Df = 5.75 # ft
Wf = 5 # ft

Cn_beta_fus = -1.3 * V / Sw / b * Df / Wf

print(f"Yaw static stability derivative for fuselage at cruise: {Cn_beta_fus:.4f} 1/rad")

Yaw static stability derivative for fuselage at cruise: -0.1410 1/rad


## Tail

The tail contribution towards yawing moment can be computed as (equation 16.39, Raymer)

$$
    C_{n_{\beta_v}} = C_{F_{\beta_v}} \frac{\partial \beta_v}{\partial \beta} \eta_v \frac{S_v}{S_w}(\bar{X}_{ac_v} - \bar{X_{cg}}),
$$

where $C_{F_{\beta_v}}$ is the side-force curve slope for vertical tail (similar to lift curve slope for horizontal tail) and is computed using the lift curve slope equation described in [aerodynamics section](../aerodynamics/lift_curve.ipynb). The $S_v$ and $S_w$ are vertical tail area and wing reference area, respectively. The $\bar{X}_{ac_v}$ denotes normalized x-location of the aerodynamic center of vertical tail. Note that $\bar{X}_{cg}$ should be most aft CG since it yields smallest moment arm.

> __*NOTE*__: The $\bar{X}$ quantities in the above expression are normalized using wing span $b$

The $\eta_v$ and $\partial \beta_v/\partial \beta$ are analogous to $\eta_h$ and $\partial \alpha_h/\partial \alpha$, respectively, and their product is computed using

$$
    \frac{\partial \beta_v}{\partial \beta} \eta_v = 0.724 + 3.06\frac{S_{vs}}{S_w} - 0.4\frac{Z_{wf}}{D_f} + 0.009A_{wing},
$$

where $S_{vs}$ is the vertical tail area if it were extended till fuselage centerline, $Z_{wf}$ is the vertical distance of the wing above the fuselage centerline, and $A_{wing}$ is the wing aspect ratio. The $S_{vs}$ is assumed to be same as $S_v$ since fuselage centerline is close to vertical tail attachment location. Below code block computes $C_{n_{\beta_v}}$:

In [17]:
# Parameters
A_vt = 1.55 * 1.47 # factor of 1.55 accounts for fusealge and HT
Sv = 17 # sq ft
Svs = 17 # sq ft
Zwf = -2.54 # ft
Xcg = 0.1861
Xac_v = 0.25 + 17/mac # vt momemt arm is 17 ft

# VT force curve slope calculation
M = 0.3
sweep_vt = 30*np.pi/180 # deg, quarter chord
beta = (1 - M**2)**0.5
eta = 1.0

numerator = 2*np.pi*A_vt
denominator = 2 + (4 + (A_vt*beta/eta)**2 * (1 + (np.tan(sweep_vt)/beta)**2) )**0.5
CFbeta_v = numerator / denominator # 1/rad

print(f"Side force curve slope for VT: {CFbeta_v:.4f} 1/rad")

factor = 0.724 + 3.06 * Svs / Sw - 0.4 * Zwf / Df + 0.009 * A

Cn_beta_vt = CFbeta_v * factor * Sv / Sw * (Xac_v - Xcg) * mac / b

print(f"Yaw static stability derivative for vertical tail at cruise: {Cn_beta_vt:.4f} 1/rad")

Side force curve slope for VT: 2.7356 1/rad
Yaw static stability derivative for vertical tail at cruise: 0.2472 1/rad


## Final $C_{n_\beta}$

The final $C_{n_\beta}$ is just the sum of all three components. The contributions from wing for yaw stability are usually small unless flying at high anlge of attack. Below code computes $C_{n_\beta}$ for the example airplane are three flight conditions:

In [15]:
Cn_beta_landing = Cn_beta_wing_landing + Cn_beta_fus + Cn_beta_vt
Cn_beta_takeoff = Cn_beta_wing_takeoff + Cn_beta_fus + Cn_beta_vt
Cn_beta_cruise = Cn_beta_wing_cruise + Cn_beta_fus + Cn_beta_vt

print(f"Yaw static stability derivative for the airplane (1/rad):")
print(f"landing: {Cn_beta_landing:.4f}")
print(f"takeoff: {Cn_beta_takeoff:.4f}")
print(f"cruise: {Cn_beta_cruise:.4f}")

Yaw static stability derivative for the airplane (1/rad):
landing: 0.1544
takeoff: 0.1385
cruise: 0.1077


For all three conditions, $C_{n_\beta}$ is positive which implies that airplane will be statically stable in yaw direction. Also, the $C_{n_\beta}$ values computed above are close to the historical data reported in Figure 16.20 in Raymer. This concludes yaw static stability estimation, next section describes roll static stability.